# 2.6 Expanding the Capability Boundaries of Q&A Bots with Plugins## 🚄 PrefaceIn the previous lessons, you have already mastered methods to improve the performance of Q&A bots by optimizing prompts and retrieval processes. However, the current Q&A bot still has certain limitations. This chapter will guide you in uncovering these shortcomings and introduce the application of agent (Agent) to address these issues. These agent are based on large language models (LLMs) and can extend the capabilities of LLMs, akin to equipping the brain with limbs.## 🍁 Course ObjectivesAfter completing this lesson, you will be able to:* Understand the core concepts of Agent systems* Familiarize yourself with the design and implementation of Multi-Agent systems* Master key tools and frameworks to handle complex tasks## 1. Limitations of Bots and SolutionsSome colleagues hope that the Q&A bot can possess a feature: by simply saying "Please submit my leave request for tomorrow," the bot would automatically submit the leave application form. However, traditional Q&A bots have inherent limitations: 

In [ ]:
# Import the required dependency packagesfrom config.load_key import load_keyimport os# Load the API keyload_key()# Do not output the API Key to logs in the production environment to avoid leakageprint(f"Your configured API Key is: {os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}")

In [ ]:
from fontTools.ttLib.tables.ttProgram import instructionsfrom chatbot import rag# The index was already built in the previous chapter, so the index can be loaded directly here. If you need to rebuild the index, you can add a line of code: rag.indexing()index = rag.load_index()query_engine = rag.create_query_engine(index=index)rag.ask('Zhang San's HR wants to take a leave tomorrow', query_engine=query_engine)

From the example above, you will notice that the current large language models (LLMs) are merely question-and-answer systems based on text input and output, incapable of interacting with the external environment.<img src="https://img.alicdn.com/imgextra/i3/O1CN01rdstgY1uiZWt8gqSL_!!6000000006071-0-tps-1970-356.jpg" alt="image" width="600" hight="300">To address this issue, you can introduce a new capability to the Q&A bot: dynamically parsing user requirements and taking corresponding actions. For instance, to enable the Q&A bot to help users request leave, you need the LLM to parse the user’s needs and call the relevant API (such as a leave application API). This is the core concept behind agent applications — through task decomposition and automated execution, agents can efficiently respond to and complete complex operations.<img src="https://gw.alicdn.com/imgextra/i4/O1CN01Xud4YB1D1UniMmUaN_!!6000000000156-0-tps-952-410.jpg" alt="image" width="600" hight="700">## 2. How to Build an AgentTypically, building an agent involves several steps, which you will follow step by step as shown in the figure below to complete the construction of an agent.<img src="https://img.alicdn.com/imgextra/i3/O1CN01wyaLNO1VE0hqPKAIm_!!6000000002620-0-tps-1083-91.jpg" alt="image" width="600">### 2.1 Define Objectives“Those who cannot plan for the whole cannot manage a part.” — In any complex task, defining objectives is the first step toward success. Just as drawing a map requires determining the destination first, when building a Q&A bot, you also need to clearly define the core goals of the task.You want the Q&A bot to be able to query employee information from the company's private database and assist users in completing leave requests while recording and updating them in the database.Thus, your first objective is: Convert all user questions about internal personnel information into tool functions for database queries. Specifically, this includes:- Converting natural language questions provided by users into corresponding SQL query statements (i.e., NL2SQL, natural language to SQL).- Using the generated SQL query statement to access the database and retrieve corresponding query results.- Returning the query results as the output of the tool function to the user.### 2.2 Define Tool FunctionsNext, after configuring the environment variables, you can start building an Agent.Of course, constructing an Agent from scratch involves handling complex low-level implementations, which often require significant time and effort. Therefore, you can use the assistant API to help you build the Agent more efficiently.The assistant API is an interface that simplifies the creation process of intelligent applications. It provides rich functionalities, including support for multiple foundational models, flexible tool invocation, dialogue management, and high scalability.Through the assistant API, you can focus on the core functionalities of the agent without having to deal with tedious low-level implementations.First, you need to define some tool functions. Assume that your Q&A bot needs to have the ability to query employee information from the database.To help you focus more on the content of the Agent, you need to simulate a query process without actually querying the database.> Assume the employee table is named 'employee', with fields including department, name, and HR.> If you wish to further enhance the performance of LLMs in NL2SQL, please visit the fine-tuning tutorial.

In [3]:
# Import dependenciesfrom llama_index.llms.dashscope import DashScopefrom llama_index.core.base.llms.types import MessageRole, ChatMessage# Define an employee query functiondef query_employee_info(query):    '''    Input user question, output employee information query result    '''    # 1. First, based on the user's question, use NL2SQL to generate SQL statements    llm = DashScope(model_name="qwen-plus")    messages = [        ChatMessage(role=MessageRole.SYSTEM, content='''You have a table called employees that records the company's employee information. This table has three fields: department, name, and HR.    You need to generate SQL statements based on user input for queries. Only generate SQL statements, no content outside of SQL statements, and do not include the ```sql``` tag.'''),        ChatMessage(role=MessageRole.USER, content=query)    ]    SQL_output = llm.chat(messages).message.content    # Print out the SQL statement    print(f'SQL statement is: {SQL_output}')    # 2. Query the database based on the SQL statement (simulated query here), and return the result    if SQL_output == "SELECT COUNT(*) FROM employees WHERE department = 'Education Department';":        return "There are 66 employees in the Education Department."    if SQL_output == "SELECT HR FROM employees WHERE name = 'Zhang San';":        return "Zhang San's HR is Li Si."    if SQL_output == "SELECT department FROM employees WHERE name = 'Wang Wu';":        return "Wang Wu's department is the Logistics Department."    else:        return "Sorry, I can't answer your question at the moment."# Test this functionquery_employee_info("How many people are in the Education Department?")

SQL语句为：SELECT COUNT(*) FROM employees WHERE department = '教育部门';


'教育部门共有66名员工。'

### 2.3 Integrate Tool Functions with Large Language Models into the AgentYou have already defined the tool functions, and the next step is to integrate them with large language models (LLMs) (LLMs) into the Agent through the Assistant API.Using the Assistants.create method, you can create a new Agent and define it using parameters such as model (model name), name (Agent naming), description (Agent's descriptive information), instructions (a crucial parameter for the Agent, used to indicate the capabilities of the tool functions while also standardizing the output format), and tools (tool functions are passed in via this parameter).> Among these, the function.name in the tools parameter is used to specify the tool function, but it needs to be in string format; therefore, it can be mapped to the tool function using a map method.

In [ ]:
# Introduce dependenciesfrom dashscope import Assistants, Messages, Runs, Threadsimport json# Define the company assistantChatAssistant = Assistants.create(    # Specify the model name here    model="qwen-plus",    # Specify the Agent name here    name='Company Assistant',    # Specify the description of the Agent here    description='An intelligent assistant capable of querying employee information, helping employees submit leave requests, or querying company rules and regulations.',    # Used to prompt the capabilities of the large model's tool functions, and can also standardize the output format    instructions='''You are the company assistant, and your functions include the following three:    1. Query employee information. For example: query who is the HR for employee Zhang San;    2. Send leave requests. For example: when an employee wants to request leave, you can help them send the leave application in the system;    3. Query company rules and regulations. For example: what is the tool used for project management in our company?    Please accurately determine which tool needs to be called and politely answer the user's questions.    ''',    # Pass in the tool functions    tools=[        {            # Define the type of tool function, generally set to function            'type': 'function',            'function': {                # Define the name of the tool function, mapped to the query_employee_info function via the map method                'name': 'Query Employee Information',                # Define the description of the tool function, the Agent mainly determines whether to call this tool function based on the description                'description': 'Very useful when querying employee information, such as querying who is the HR for employee Zhang San, querying the total number of people in the education department, etc.',                # Define the parameters of the tool function                'parameters': {                    'type': 'object',                    'properties': {                        # Use the user's question as the input parameter                        'query': {                            'type': 'str',                            # Description of the input parameter                            'description': 'The user's question.'                        },                    },                    # Declare which parameters are required for this tool function here                    'required': ['query']},            }        }    ])print(f'{ChatAssistant.name} creation completed')# Establish the mapping relationship between Agent Function name and tool functionsfunction_mapper = {    "Query Employee Information": query_employee_info}print('Mapping relationship between tool functions and function.name established')

At the same time, you can encapsulate a helper function `get_agent_response`.The functionality of this code is: when a user sends a request to the Agent, the Agent uses get_agent_response() to send the request and obtain a response. If the task requires calling an external tool (such as a database query), the Agent will execute corresponding operations based on the mapping of the tool functions and return the result to the user. This allows the Agent to handle more complex tasks rather than just simple question-and-answer interactions.The process of obtaining an Agent's response through the Assistant API involves concepts such as thread, message, and run. If you are interested in these concepts and details, please refer to the [Alibaba Cloud Assistant API Official Documentation](https://help.aliyun.com/zh/model-studio/developer-reference/assistantapi/).> If you wish to equip the Agent with more capabilities, you can add tool functions and establish a mapping relationship in function_mapper and tools.

In [ ]:
# Input message information and output the response from the specified Agentdef get_agent_response(assistant, message=''):    # Create a new session thread    thread = Threads.create()        # Create a message and send it to the session thread    message = Messages.create(thread.id, content=message)        # Create a run instance (run request), associating the session thread with the Assistant (agent)    run = Runs.create(thread.id, assistant_id=assistant.id)        # Wait for the run to complete, checking if the task is finished    run_status = Runs.wait(run.id, thread_id=thread.id)        # If the task run fails, output the error message    if run_status.status == 'failed':        print('run failed:')            # If tools are needed to assist the model in operations (e.g., querying a database, sending requests, etc.)    if run_status.required_action:        # Get detailed information about the tool function to be called        f = run_status.required_action.submit_tool_outputs.tool_calls[0].function                # Get the name of the tool function (function name)        func_name = f['name']                # Get the input parameters required for calling the tool function        param = json.loads(f['arguments'])                # Print out the tool's name and parameter information        print("function is", f)                # Based on the tool function's name, find the corresponding actual tool function through a mapping (function_mapper)        # Here, a dictionary mapping (function_mapper) is used, which maps tool function names to specific functions        if func_name in function_mapper:            # Use the mapping to find the actual tool function and pass the parameters to get the result            output = function_mapper[func_name](**param)        else:            # If no corresponding function is found, output is empty            output = ""                # Prepare to submit the output (result) of the tool function        tool_outputs = [{            'output': output        }]                # Submit the output results back to the run instance        run = Runs.submit_tool_outputs(run.id, thread_id=thread.id, tool_outputs=tool_outputs)                # Wait for the run to complete        run_status = Runs.wait(run.id, thread_id=thread.id)        # Get the final run result    run_status = Runs.get(run.id, thread_id=thread.id)        # Get the list of message records    msgs = Messages.list(thread.id)        # Return the Agent's reply content    return msgs['data'][0]['content'][0]['text']['value']

### 2.4 Try ConversingYou have completed the construction of a simple single-Agent system. Testing is an essential step before it is officially put into use. You can try conversing with the Q&A bot:  

In [ ]:
query_stk = [    "Who is Zhang San's HR?",    "How many employees are there in the education department?",    "Which department is Wang Wu in?",]for query in query_stk:    print("The question is:")    print(query)    print("Thought process and final output:")    print(get_agent_response(ChatAssistant, query))    print("\n")

As can be seen from the test results, the Q&A bot with extended functionality has achieved the expected effect.<img src="https://img.alicdn.com/imgextra/i4/O1CN01q6NyMr237JtdJFO8Q_!!6000000007208-2-tps-2082-974.png" alt="image" width="600" height="300">In practical applications, agents can not only interact with the outside world but also enhance their ability to handle complex tasks through different modular designs. The working principle of agents can be understood through the following core modules:- Tool Module    The tool module is responsible for defining and managing the various tools that the agent can use. This includes the description, parameters, and functional characteristics of the tools. This module ensures that the agent can understand and effectively use these tools to complete tasks.- Memory Module    The memory module can be divided into long-term memory and short-term memory.    Long-term memory is used to store persistent information and experiences, helping the agent with pattern learning, knowledge accumulation, and personalized services.    Short-term memory is used to temporarily store information related to the current task, supporting the agent in real-time decision adjustments during task execution.- Planning Ability    The planning ability module is responsible for task planning. Through the agent's decision-making ability, this part helps the agent break down complex tasks, formulate specific action steps and strategies, ensuring the successful completion of tasks.- Action Ability    Action ability works closely with the tool module to ensure that the agent can select the appropriate tools and execute corresponding operations through containers. Action ability is the core of the agent’s task implementation, ensuring that it can effectively carry out tasks according to predetermined plans and decisions.Through the collaboration of these modules, agents can handle complex tasks, improve the efficiency and accuracy of task execution, and break through the limitations of traditional methods.If you are interested in these concepts, please refer to [Alibaba Cloud Large Language Model ACA Course](https://edu.aliyun.com/course/3126500/lesson/342570389?spm=a2cwt.28196072.ACA13.12.660e6e7b6G8Ye7) and the extended reading section of this chapter.

## 3. multi-agentAfter completing the query of employee information, you will next need to apply for and record employees' leave requests. Therefore, you need to add a new tool function to meet this requirement.This tool function takes the leave date entered by the employee as an input parameter and returns a string indicating successful application. To help you focus more on the content related to agent, the example below simulates the leave application process without actually submitting the leave request to the company system.  

In [ ]:
def send_leave_application(date):    '''    Enter the leave time, output the result of the leave application    '''    return f'The leave application has been sent for you. The leave date is {date}.'# Test this functionprint(send_leave_application("The day after tomorrow"))

After confirming that the new tool function is working properly, you need to integrate this new function into the agent you created earlier:  

In [ ]:
new_tool = {'type': 'function',            'function': {                'name': 'Send Leave Application',                'description': 'Very useful when you need to help employees send leave applications.',                'parameters': {                    'type': 'object',                    'properties': {                        # Time for leave request                        'date': {                            'type': 'str',                            'description': 'The time when the employee wants to request leave.'                        },                    },                    'required': ['date']},            }           }ChatAssistant.tools.append(new_tool)function_mapper["Send Leave Application"] = send_leave_applicationprint('Mapping relationship between leave tool function and function.name established')

After confirming the integration is successful, you can test the model's output to ensure everything is functioning properly:  

In [ ]:
get_agent_response(ChatAssistant, "Who is Zhang San's HR? Request three days of leave for him")

From the output above, you will find that when handling complex tasks, especially when a robot needs to perform multiple operations within a single request, a single Agent may not be able to effectively complete all subtasks.For example, the user request “Who is Zhang San’s HR? Request three days of leave for him” involves two operations: employee information lookup and leave application. A single Agent typically can only handle one type of task and cannot simultaneously invoke multiple tools or API interfaces to complete all subtasks.To overcome this limitation of multi-operation requirements, you can introduce a new capability to the Q&A bot: breaking down tasks into multiple independent modules for processing. A multi-agent system is specifically designed for this purpose.The multi-agent system overcomes the limitation of a single Agent being unable to perform multiple operations at once by breaking down tasks into multiple subtasks, which are then handled by different Agents. Each Agent focuses on a specific task, functioning like a member of a team, each with their own responsibilities, ultimately collaborating to complete the entire task.This design not only improves task-handling efficiency but also enhances flexibility, ensuring that each subtask receives specialized processing.There are various design approaches for Multi-Agent systems. This tutorial will introduce a Multi-Agent system consisting of a Planner Agent, several Agents responsible for executing tool functions, and a Summary Agent.- Planner Agent:    Based on the user's input, selects which Agent or combination of Agents should complete the task.- Tool Function Execution Agents:    Execute their respective tool functions based on the tasks distributed by the Planner Agent.- Summary Agent:    Generates a summary based on the user's input and the outputs from the Tool Function Execution Agents, then returns it to the user.<a href="https://img.alicdn.com/imgextra/i1/O1CN01kX9Asm1dNdxmW18nG_!!6000000003724-2-tps-2074-935.png" target="_blank"><img src="https://img.alicdn.com/imgextra/i1/O1CN01kX9Asm1dNdxmW18nG_!!6000000003724-2-tps-2074-935.png" alt="image" width="800" height="400"></a>Returning to the previous example—“Who is Zhang San’s HR? Request three days of leave for him.” In a multi-agent system, this task would be broken down into two subtasks:Querying Zhang San’s HR information: Handled by one Agent.Submitting the leave application: Handled by another Agent.Through the multi-agent system, the Planner Agent first analyzes the user request and breaks it down into these two subtasks, then assigns each task to the corresponding execution Agent for processing. Finally, the Summary Agent aggregates the results from the various Agents and generates the final response.### 3.1 Planner AgentThe Planner Agent is the core component of the Multi-Agent system. It is responsible for analyzing problems and deciding which Agent or combination of Agents should receive the task.First, use the Assistant API to create the Planner Agent. Here, you do not need to specify instructions for now:

In [ ]:
# Agent at the decision-making level, determining which agents to use and their execution order.planner_agent = Assistants.create(    model="qwen-plus",    name='Process Orchestration Robot',    description='You are the team leader with many agents under you. You need to decide the order of using these agents based on user input.')print("Planner Agent created successfully")

After creation, you can first take a look at the output of the Planner Agent when instructions are not defined:  

In [ ]:
print(get_agent_response(planner_agent,'Who is Zhang San's HR? How many employees are there in the education department?'))

From the robot's response, you can see that its output contains a lot of extra information and does not specify the name of the Agent to be invoked.Next, you need to use instructions to specify the Agents it can invoke and the output format.Currently, you need the robot to help employees with internal company employee information queries, leave applications, and other daily conversations.Additionally, since the Agent's return value is in string format, it will be inconvenient when you later need to call the corresponding intelligent agents based on the returned content. Therefore, you need to require the Planner Agent to output a list-formatted string. For example: `['employee_info_agent', 'leave_agent', 'company_info_agent']`, so that subsequent string parsing tools can convert it into structured data.

In [ ]:
planner_agent=Assistants.update(planner_agent.id,instructions="""You have the following agents in your team.\n    employee_info_agent: Can query the company's employee information. If the question is about department, HR, or related information, call this agent;\n    leave_agent: Can help employees send leave requests. If the user requests leave, call this agent;\n    chat_agent: If the user's question does not require any of the above agents, call this agent.\n\n    You need to determine the order in which to use these agents based on the user's question. An agent can be called multiple times. Your return format is a list, and you cannot return any other information. For example: ["employee_info_agent", "leave_agent"] or ["chat_agent"], where the elements in the list can only be the agents mentioned above.""")print("Instructions for Planner Agent have been updated")

Next, try a few test questions to see if the Planner Agent can distribute them to the correct Agent.  

In [ ]:
query_stk = [    "Who is Zhang San's HR? How many employees are there in the education department?",    "Which department is Wang Wu in? Help me submit a leave application for next Wednesday",    "Hello"]for query in query_stk:    print("The question is:")    print(query)    print(get_agent_response(planner_agent,query))    print("\n")

For these three test questions, the Planner Agent made the correct choices.You can observe that when the Planner Agent returns the task planning result, its output is a list-form string describing the task execution order, such as: ["employee_info_agent", "leave_agent"]. To facilitate subsequent processing and execution, you need to convert it into Python's native list structure (list) while maintaining the corresponding call order. Here, you can use Python’s ast.literal_eval method, which can safely parse string expressions into corresponding Python data types, such as lists, dictionaries, etc.In this way, you can transform the task planning results into an easily operable list object and gradually parse out each task's execution steps to simplify subsequent multi-agent collaboration.

In [ ]:
import ast# Use Planner Agent to get task planningplanner_response = get_agent_response(planner_agent, "Which department is Wang Wu in? Help me submit a leave application for next Wednesday.")# Parse the string format response from Planner Agent into a list# Planner Agent returns a list-formatted string describing the invocation order, e.g.: ["employee_info_agent", "leave_agent"]order_stk = ast.literal_eval(planner_response)# Print out the planning result of Planner Agentprint("Planner Agent's task planning result:")for i, agent in enumerate(order_stk, start=1):    print(f'Step {i}: Invoke {agent}')

### 3.2 Agent for Executing Tool FunctionsIn the previous chapter, you have completed the planning work of the Planner Agent. It is like the queen ant in an ant colony, capable of coordinating tasks and issuing commands. However, relying solely on the queen ant is not enough to make the entire colony function — countless worker ants are needed to carry out specific tasks, such as gathering food or building nests. Similarly, in your multi-agent system, the Planner Agent alone is insufficient to complete tasks. It must be equipped with an Agent responsible for executing tool functions to truly achieve efficient collaboration across the entire system.Next, based on the planning results from the previous section, you will need to create separate Agents for executing tool functions for two different tasks, each responsible for specific operational tasks. This design not only modularizes the system but also maximizes the coordination capabilities of the Planner Agent.> Ensure that the agent variable names match those defined in the Planner Agent's instructions.

In [ ]:
# Employee information query agentemployee_info_agent = Assistants.create(    model="qwen-plus",    name='Employee Information Query Assistant',    description='An intelligent assistant capable of querying employee information.',    instructions='''You are the employee information query assistant, responsible for querying employee names, departments, HR, and other information''',    tools=[        {            'type': 'function',            'function': {                'name': 'Query Employee Information',                'description': 'Very useful when you need to query employee information, such as querying whose HR Zhang San is or the total number of people in the education department.',                'parameters': {                    'type': 'object',                    'properties': {                        'query': {                            'type': 'str',                            'description': 'The user's question.'                        },                    },                    'required': ['query']},            }        }    ])print(f'{employee_info_agent.name} created')# Leave application agentleave_agent = Assistants.create(    model="qwen-plus",    name='Leave Application Assistant',    description='An intelligent assistant that helps employees submit leave applications.',    instructions='''You are the employee leave application assistant, responsible for helping employees submit leave applications.''',    tools=[        {            'type': 'function',            'function': {                'name': 'Send Leave Application',                'description': 'Very useful when you need to help employees send leave applications.',                'parameters': {                    'type': 'object',                    'properties': {                        # Time for which leave is required                        'date': {                            'type': 'str',                            'description': 'The time when the employee wants to take leave.'                        },                    },                    'required': ['date']},            }        }    ])print(f'{leave_agent.name} created')# Functionality is to reply to daily questions. For daily questions, a lower-cost model can be used as the base for the agent.chat_agent = Assistants.create(    # Since this agent does not require high performance from large models, the cost-effective qwen-turbo model is used.    model="qwen-turbo",    name='Daily Question Answering Robot',    description='An intelligent assistant that answers users' questions',    instructions='Please answer users' questions politely')print(f'{chat_agent.name} created')

### 3.3 Create Summary Agent and Test Multi-Agent EffectsAfter completing the creation of the Planner Agent and the execution tool function Agent, you also need to create the Summary Agent. This Agent will provide a comprehensive and complete answer to the user's question based on the user's query and the reference information output by previous Agents.

In [ ]:
summary_agent = Assistants.create(    model="qwen-plus",    name='Summary Bot',    description='An intelligent assistant that comprehensively and completely answers user questions based on the provided information',    instructions='You are an intelligent assistant that comprehensively and completely answers user questions based on the provided information')print(f'{summary_agent.name} created successfully')

You can encapsulate the above steps into a function called get_multi_agent_response, which abstracts the complex multi-agent collaboration process into a simple interface. With this encapsulation, users only need to provide the input question, and the function will:1. Call the Planner Agent to plan the task sequence.2. Sequentially invoke the corresponding tool function Agents according to the planned order to execute tasks.3. Aggregate all task results and finally generate the final response through the Summary Agent.This design not only makes the main process clearer but also facilitates reuse and scalability.> Since the elements in the list are strings, an agent_mapper method is used to map the string-formatted Agents to predefined Agent objects.

In [ ]:
# Map the strings in the list to Agent objects# Map string-formatted Agent names to specific Agent objectsagent_mapper = {    "employee_info_agent": employee_info_agent,    "leave_agent": leave_agent,    "chat_agent": chat_agent}def get_multi_agent_response(query):    # Get the execution order of Agents    agent_order = get_agent_response(planner_agent, query)    # Since the output of large models may be unstable, an exception handling module is added to handle issues with parsing list strings.    try:        order_stk = ast.literal_eval(agent_order)        print("Planner Agent is working:")        for i in range(len(order_stk)):            print(f'Step {i+1} calls: {order_stk[i]}')        # As more Agents are added, their outputs need to be appended to the user's question as reference information.        cur_query = query        Agent_Message = ""        # Execute Agents sequentially        for i in range(len(order_stk)):            cur_agent = agent_mapper[order_stk[i]]            response = get_agent_response(cur_agent, cur_query)            Agent_Message += f"*{order_stk[i]}* replied: {response}\n\n"            # If the current Agent is the last one, use its output as the Multi-Agent output            if i == len(order_stk)-1:                prompt = f"Refer to the known information: {Agent_Message}, and answer the user's question: {query}."                multi_agent_response = get_agent_response(summary_agent, prompt)                print(f'Multi-Agent reply: {multi_agent_response}')                return multi_agent_response            # If the current Agent is not the last one, append the previous Agent's output (response) to the next round's query as reference information            else:                # Add special markers before and after the reference information to prevent the large model from confusing it with the question                cur_query = f"You can refer to the known information: {response}. You must fully answer the user's question. The question is: {query}."    # Fallback strategy: If the above program fails, directly call the large model    except Exception as e:        return get_agent_response(chat_agent, query)# Here we test with "Which department is Wang Wu in? Help me submit a leave application for next Wednesday."get_multi_agent_response("Which department is Wang Wu in? Help me submit a leave application for next Wednesday.")

## 4. Multi-Agent Orchestration Functionality of the Large Language Model PlatformIn the previous section, you learned about the design philosophy and implementation methods of the Multi-Agent system. It can be seen that while building a Multi-Agent system independently provides a high degree of flexibility, it also comes with a certain amount of workload and complexity. For many enterprises, quickly implementing complex business logic is more important.[Dify](https://Dify.ai) is one of the pioneers of the agent workflow canvas. Dify's canvas tool allows users to design and manage the execution logic of agent tasks through a visual flowchart, setting a benchmark in the industry. Its intuitive interface design has provided a reference for many developers.Although Dify.ai excels in agent workflow orchestration, in comparison, the Model Studio is more suitable for domestic enterprises and developers.<img src="https://img.alicdn.com/imgextra/i4/O1CN01lbXhaX1WpOeXxkHwi_!!6000000002837-2-tps-1672-944.png" alt="image" width="700">The multi-agent orchestration functionality of the Model Studio leverages the powerful capabilities of large language models (LLMs), supporting autonomous decision-making and task allocation by agent. Through the platform’s built-in canvas-style orchestration tool, users can easily achieve the following goals:- Define the execution logic of agents: Users can intuitively configure the execution rules and logic chains of each agent using the canvas-style tool.- Orchestrate collaboration between multiple agents: Through modular design, different agents can efficiently cooperate to complete complex tasks.- Quickly validate business outcomes: The built-in simulation and testing functions help users rapidly verify whether the orchestration logic meets expectations.The Model Studio supports users in designing an agent system from scratch, even without a deep technical background. Users can quickly build and validate a multi-agent collaboration system through its user-friendly interface. For enterprises that need to quickly implement multi-agent technology, the Model Studio is an ideal choice.For more details, please refer to [Model Studio Agent Application](https://help.aliyun.com/zh/model-studio/user-guide/single-agent-application).## 5. Summary of This ChapterIn this lesson, you have learned the following content:- The core concept of the Agent system- The construction principles and implementation methods of multi-agent systems- Multi-agent orchestration of large language model platforms### Further ReadingYou have already learned how to use multi-agent systems to handle complex tasks, but this design is not limited to basic operations. In fact, properly designing a multi-agent system can help automate more complex business processes, improving efficiency and accuracy, and even handling tedious operations that usually require human intervention.##### Effect Demonstration and Flowchart AnalysisBelow is an actual application case of an agent system. Suppose you need to perform content validation on a webpage—not just checking static content, but also interacting with page elements. At this point, the advantages of a multi-agent system become apparent. Through division of labor and cooperation, multiple agents can efficiently complete tasks together without requiring manual intervention at each step.<a href="https://gw.alicdn.com/imgextra/i1/O1CN01w0oXM41YyfuOvSLsR_!!6000000003128-2-tps-1567-668.png" target="_blank"><img src="https://gw.alicdn.com/imgextra/i1/O1CN01w0oXM41YyfuOvSLsR_!!6000000003128-2-tps-1567-668.png" alt="image" width="700"></a>When facing a task that requires parsing HTML pages and performing specific actions, the division of labor among multiple agents is as follows:- Planner Agent: Decomposes tasks, such as identifying lists or buttons within HTML elements.- Selector Agent: Responsible for specific operational tasks, such as selecting specific elements and executing click actions.- Monitor Agent: Monitors task execution in real time to ensure the process completes as planned, such as verifying whether the correct button was clicked.##### Integration of Visual Annotations in Agent ApplicationsIn practical projects, especially those involving webpage content interaction, relying solely on traditional text analysis capabilities often cannot accurately identify interface elements, leading to decreased execution efficiency and accuracy of agents. To address this issue, visual annotations are applied to automated testing and element recognition in console interfaces.In some scenarios, when agents operate console interfaces, due to the complexity of interface elements, models may struggle to accurately identify targets. By introducing visual annotation technology, the model's understanding and operation accuracy of interface elements can be improved.Just like when you use software, you may need to visually identify buttons, drop-down menus, or forms to make the correct operation. Without clear visual identifiers, you might select the wrong location or perform incorrect actions. Similarly, accurate identification of interface elements is crucial for agents during task execution. Through visual annotations, the system can "tag" these elements, helping agents "see" them more accurately and execute operations correctly.For example, when performing automated testing on HTML pages, by using visual annotations, the system can label buttons, links, or forms on the page and guide agents to precisely select and operate these elements. This visually enhanced design significantly improves the model's ability to handle complex interfaces, making multi-agent systems more flexible and efficient when dealing with various UI elements.##### Flexibility and Technical Integration of Multi-Agent SystemsThrough this case, you can see that the design of multi-agent systems is not limited to a specific type of technology or scenario. Although visual annotations were combined with multi-agent systems in this example, the flexibility of multi-agent systems enables them to integrate with various technologies to meet different needs.For instance, you can combine machine learning technology to optimize the decision-making process of agents, or use image processing technology to enhance the recognition capability of interface elements.Alternatively, you can integrate multi-agent systems with natural language processing so that they can understand and generate more complex language instructions, applicable in intelligent customer service or text analysis scenarios.You can freely choose the architecture and technical combination of multi-agent systems based on your requirements, ultimately bringing you the best automation solution.A multi-agent system is not only a tool but also a way of thinking, capable of flexibly designing and adjusting according to different tasks and scenarios, thereby delivering significant value across various applications. In future applications, whether facing automated webpage testing or handling more complex business processes, a well-designed multi-agent system can help improve efficiency, optimize operations, and even replace some human work, freeing up productivity.## 🔥 6. Post-Class Quiz### 🔍 Single Choice Question<details><summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;"><b>What is the role of the Planner Agent in this tutorial? ❓</b>- A. Responsible for executing tool functions- B. Responsible for summarizing the output of Agents- C. Responsible for analyzing user input and distributing tasks to other Agents- D. Responsible for printing key logs**[Click to View Answer]**</summary><div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">✅ **Reference Answer: C**  📝 **Analysis**:  - In this tutorial, the main function of the Planner Agent is to act as a task distributor. It analyzes user input and assigns different tasks to the most suitable other Agents based on specific task requirements.- Therefore, the Planner Agent plays a routing role in the system, ensuring that each task is efficiently handed over to the corresponding Agent for processing.</div></details>